# Hierarchical Indexing

3D and 4D data can be handled in Python via the inbuild `Panel` and `Panel4D` objects

Hierarchical indexing a more common approach to this using multiple index levels within a single index.

Can be accessed using `Series` and `DataFrame` objects

In [ ]:
import pandas as pd
import numpy as np

## A Multiple Indexed `Series`

2D data within a 1D object

The bad way:

In [ ]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]

populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]

pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

With this indexing scheme, you can straightforwardly index or slice the series based on this multiple index:

In [ ]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [ ]:
pop.index[0]

('California', 2000)

How can we access all values from 2010?

In [ ]:
# this is not nice

pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

## Pandas to the rescue...

Our tuple-based indexing is essentially a rudimentary multi-index, and the Pandas `MultiIndex` type gives us the type of operations we wish to have:

In [ ]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [ ]:
index.levels

FrozenList([['California', 'New York', 'Texas'], [2000, 2010]])

Notice that the `MultiIndex` contains multiple levels of indexing–in this case, the state names and the years

Re-indexing results in the following:

In [ ]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [ ]:
pop.index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [ ]:
type(pop)

pandas.core.series.Series

Back to our original question...

In [ ]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [ ]:
pop['California', :]

2000    33871648
2010    37253956
dtype: int64

In [ ]:
pop['California']

2000    33871648
2010    37253956
dtype: int64

## `MultiIndex` as an extra dimension

The previous slides look familiar: why not just store this information with some extra indices and column labels?

In [ ]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [ ]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

So, why are we doing this?

It gives us flexibility in types of data that we can represent.

Example: adding an additional column of demographic data for each state for each year:

In [ ]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [ ]:
pop_df.index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

We can use ufuncs on hierarchical indices too:

In [ ]:
f_u18 = pop_df['under18'] / pop_df['total']
print(f_u18)
f_u18.unstack()

California  2000    0.273594
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64


,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


This allows us to easily and quickly manipulate and explore even high-dimensional data.

## Methods of MultiIndex Creation

Simply pass a list of two or more index arrays to the constructor of a `Series` or `DataFrame` object:

In [ ]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.497840  0.424252
  2  0.404516  0.295399
b 1  0.566983  0.697385
  2  0.705312  0.949341

In [ ]:
df.unstack()

data1               data2          
          1         2         1         2
a  0.497840  0.404516  0.424252  0.295399
b  0.566983  0.705312  0.697385  0.949341

You can pass a dictionary with tuples as keys:

In [ ]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

We can also give the levels of the MultiIndex names to keep track of the meaning of indices:

In [ ]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [ ]:
pop.unstack()

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


## MultiIndex for Columns

In a `DataFrame`, the rows and columns are completely symmetric, and just as the rows can have multiple levels of indices, the columns can have multiple levels as well.

In [ ]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
print(data)
data[:, ::2] *= 10
print(data)
data += 37
print(data)

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

[[-0.8 -0.8  0.4 -0.6  0.4  0. ]
 [-1.9 -0.2  2.1 -0.4  0.7  0.3]
 [ 1.9  0.5 -0.4  0.6  1.2 -0.7]
 [ 1.   0.4  0.2  1.3 -1.3  0.1]]
[[ -8.   -0.8   4.   -0.6   4.    0. ]
 [-19.   -0.2  21.   -0.4   7.    0.3]
 [ 19.    0.5  -4.    0.6  12.   -0.7]
 [ 10.    0.4   2.    1.3 -13.    0.1]]
[[29.  36.2 41.  36.4 41.  37. ]
 [18.  36.8 58.  36.6 44.  37.3]
 [56.  37.5 33.  37.6 49.  36.3]
 [47.  37.4 39.  38.3 24.  37.1]]


subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      29.0  36.2  41.0  36.4  41.0  37.0
     2      18.0  36.8  58.0  36.6  44.0  37.3
2014 1      56.0  37.5  33.0  37.6  49.0  36.3
     2      47.0  37.4  39.0  38.3  24.0  37.1

This is where multi-indexing comes in handy. With this in place, we can now observe the records for a given subject over a given time-period:

In [ ]:
health_data['Bob']['Temp']

year  visit
2013  1        36.2
      2        36.8
2014  1        37.5
      2        37.4
Name: Temp, dtype: float64

## How can we index and slice a MultiIndex?

Let's look at this for a `Series` first:

In [ ]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [ ]:
pop['California', 2000]

33871648

In [ ]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [ ]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [ ]:
pop.loc[('California', 2005):('New York')]

state       year
California  2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [ ]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [ ]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [ ]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

In [ ]:
pop['California', 'Texas']

KeyError: ignored

How about for a `DataFrame`?

In [ ]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      29.0  36.2  41.0  36.4  41.0  37.0
     2      18.0  36.8  58.0  36.6  44.0  37.3
2014 1      56.0  37.5  33.0  37.6  49.0  36.3
     2      47.0  37.4  39.0  38.3  24.0  37.1

In [ ]:
health_data['Guido', 'HR']

year  visit
2013  1        41.0
      2        58.0
2014  1        33.0
      2        39.0
Name: (Guido, HR), dtype: float64

In [ ]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      29.0  36.2
     2      18.0  36.8

In [ ]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        29.0
      2        18.0
2014  1        56.0
      2        47.0
Name: (Bob, HR), dtype: float64

## Rearranging Multi-Indices

One of the keys to working with multiply indexed data is knowing how to effectively transform the data.

Let's start by creating some simple multiply indexed data where the indices are not lexographically sorted:

In [ ]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.305291
      2      0.749270
c     1      0.249740
      2      0.874213
b     1      0.968938
      2      0.319291
dtype: float64

In [ ]:
data['a':'b']

UnsortedIndexError: ignored

In [ ]:
# need to sort the index prior to slicing
data = data.sort_index()
data

char  int
a     1      0.305291
      2      0.749270
b     1      0.968938
      2      0.319291
c     1      0.249740
      2      0.874213
dtype: float64

In [ ]:
data['a':'b']

char  int
a     1      0.305291
      2      0.749270
b     1      0.968938
      2      0.319291
dtype: float64

## Stacking and unstacking indices

It is possible to convert a dataset from a stacked multi-index to a simple two-dimensional representation, optionally specifying the level to use:

In [ ]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [ ]:
pop.unstack()

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [ ]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [ ]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [ ]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

## Data Aggregations on Multi-Indices

We've previously seen that Pandas has built-in data aggregation methods, such as `mean()`, `sum()`, and `max()`.

For hierarchically indexed data, these can be passed a level parameter that controls which subset of the data the aggregate is computed on:

In [ ]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      29.0  36.2  41.0  36.4  41.0  37.0
     2      18.0  36.8  58.0  36.6  44.0  37.3
2014 1      56.0  37.5  33.0  37.6  49.0  36.3
     2      47.0  37.4  39.0  38.3  24.0  37.1

In [ ]:
data_mean = health_data.groupby(level='year').max()
data_mean

subject   Bob       Guido         Sue      
type       HR  Temp    HR  Temp    HR  Temp
year                                       
2013     29.0  36.8  58.0  36.6  44.0  37.3
2014     56.0  37.5  39.0  38.3  49.0  37.1

In [ ]:
data_mean = health_data.groupby(level='year').mean()
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     23.5  36.50  49.5  36.50  42.5  37.15
2014     51.5  37.45  36.0  37.95  36.5  36.70

Can take the mean along levels of the columns too

In [ ]:
data_mean.groupby(axis=1, level='type').mean()

type,HR,Temp
year,,
2013,38.500000,36.716667
2014,41.333333,37.366667


In [85]:
print(data_mean.index.levels)

AttributeError: ignored

In [ ]:
data_mean.groupby(axis=0, level='subject').mean()

ValueError: ignored

# Summary

By allowing multi-indexing we enable a powerful querying mechanism over the data that we are using.

